# EDA To Prediction (DieTanic) 커널 해석


### *Sometimes life has a cruel sense of humor, giving you the thing you always wanted at the worst time possible.*  
-Lisa Kleypas

                                                                                                                                     

타이타닉의 침몰은 역사상 가장 악명 높은 침몰중 하나이다. 1912년 4월 15일에 첫 항해에서, 타이타닉은 빙산과 충돌한 뒤 가라앉았고, 2224명의 승객중 1502명의 승객이 사망하였다. 이것이 **DieTanic**라고 명칭한 이유이다. 이것은 전 세계에서 누구도 잊을 수 없는 재해이다.

타이타닉을 만드는데에는 약 750만 달러가 쓰였고 충돌때문에 바다 아래로 가라앉았다. 타이타닉 데이터셋은 데이터 과학을 시작하고 캐글 컴피티션에 참가하려는 초보자에게 아주 좋은 데이터 셋이다.

이 notebook의 목적은 예측 모델링 문제에서 작업흐름이 어떻게게 흘러가는지에 대한 아이디어를 제공해준다. feature를 확인하는 방법, 새로운 features를 더하는 방법, 그리고 몇개의 Machine Learning 개념들.... 나는 새로운 초보자들도 그것의 흐름을 이해할수 있도록 notebook을 가능한 기본적인 수준으로 만들려고 노력했다.  

만약 당신이 notebook 이 좋고 이것이 도움이 됬다면 **PLEASE UPVOTE**. 이것은 글쓴이에게 큰 동기부여가 됩니다.

## Contents of the Notebook:

#### Part1: 탐색적 데이터 분석Exploratory Data Analysis(EDA):
1)feature 분석(Analysis of the features).

2)다양한 feature를 통해 관계나 trends 찾기
(Finding any relations or trends considering multiple features.)

#### Part2: Feature Engineering and Data Cleaning:
1)새로운 features를 더하기(Adding any few features).

2)불필요한 feature 제거(Removing redundant features).

3)modeling을 위해 feature를 적절한 형태로 변환
(Converting features into suitable form for modeling).

#### Part3: Predictive Modeling

1)기본적인 알고리즘(Running Basic Algorithms).

2)교차검증(Cross Validation).

3)앙상블(Ensembling).

4)중요한 feature 추출(Important Features Extraction).

## Part1: Exploratory Data Analysis(EDA)

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df_test = pd.read_csv('../input/test.csv')

In [ ]:
test_id = df_test.PassengerId

In [ ]:
data=pd.read_csv('../input/train.csv')

In [ ]:
df_test.insert(loc = 1, column = 'Survived', value = np.repeat(np.nan, df_test.shape[0], axis=0), )

In [ ]:
data = pd.concat([data, df_test], axis = 0, ignore_index=True)

In [ ]:
data.isnull().sum() #각 column 별 결측치 확인

**Age, Cabin and Embarked** 는 결측치를 가진다. 이것들을 수정할 것이다.

### 얼마나 많은 사람이 살아남았나??

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True) # explode: pie 조각 떨어지는 정도 #autopct % 표시 형식
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived',data=data,ax=ax[1])
ax[1].set_title('Survived')
plt.show()

많은 승객이 사고에서 살아남지 못한것은 명확하다.

training set의 891명의 승객중 오직 약 350명정도가 살아남았다. 즉 오직 전체 training set의 **38.4%** 만 충돌에서 살아남았다. 우리는 데이터로부터 더 나은 통찰을 얻을 필요가 있으며 승객의 어떤 categories가 살아남거나 살아남지 못했는지 볼 필요가 있다.

데이터셋의 다른 feature들을 사용해서 생존률을 확인할것이다. 이 feature들은 Sex, Port of Embarcation, Age, 등등이다.

우선 feature 다른 타입들을 살펴 보자.

## Types Of Features

### Categorical Features:
A categorical variable 은 2개 이상의 범주를 가지고 있고 각각의 값은 범주화 될 수 있다. 예로,  성별은 categorical Feature로 2개의 범주를 가지고있다 (male and female). 이러한 값에 대해 어떤 순서를 매기거나 정렬을 할 수 없다. 이런 변수들은 **Nominal Variables(명목형 변수)** 로 알려져있다.

**Categorical Features in the dataset: Sex,Embarked.**

### Ordinal Features:
ordinal variable 는 categorical values와 유사하나 ordinal variable는 값들 사이에 상대적인 순서를 가지거나 정렬을 할 수 있다는 것이다. 즉 만약 **Tall, Medium, Short**의 값을 가지는 **Height**와 같은 feature 가 있다면. Height는 ordinal variable이다. 여기서 변수사이의 상대적인 순서가 있는것을 알 수 있다.

**Ordinal Features in the dataset: PClass**

### Continous Feature:
feature가 연속적이란것은 그것이 feature column의 최소값과 최대값 사이 또는 어떠한 두 값의 사이에 값을 취할 수 있다는것을 의미한다.

**Continous Features in the dataset: Age**

## Analysing The Features

## Sex--> Categorical Feature

In [ ]:
data.groupby(['Sex','Survived'])['Survived'].count()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex',hue='Survived',data=data,ax=ax[1])
ax[1].set_title('Sex:Survived vs Dead')
plt.show()

이것은 꽤 흥미롭다. 배에 승선한 남자의 수는 여자의 수보다 더 많다. 그러나 생존한 여자의 수는 생존한 남자의 수의 거의 두배이다. 배의 승선한 **여자의 생존률은 거의 75%인 반면에 남자는 약 18~19%이다.** 

이것은 modeling에 **매우 중요한** feature로 보인다. 그러나 이것이 가장 좋은 것일까? 
<br>다른 feature들을 확인해보자.

## Pclass --> Ordinal Feature

In [ ]:
pd.crosstab(data.Pclass, data.Survived, margins = True).style.background_gradient(cmap = 'terrain')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'],ax=ax[0])
ax[0].set_title('Number Of Passengers By Pclass')
ax[0].set_ylabel('Count')
#sns.barplot('Pclass', y="Survived", data=data)
sns.countplot('Pclass',hue='Survived',data=data,ax=ax[1]) #글쓴이는 수로 확인 저는 평균확인
ax[1].set_title('Pclass:Survived vs Dead')
plt.show()

사람들은 **돈으로 모든것을 살 수 없다** 라고 말한다. 그러나 Pclass 1 의 승객들이 우선적으로 구조된것을 명확하게 확인 할 수 있었다. Pclass3의 승객이 훨씬 더 많긴 하지만 살아남은 수는 매우 낮다. 약 **25%**.

Pclass 1 의 생존률은 약 **63%** 이지만 Pclass 2의 생존률은 약 **48%** 이다. 이래서 돈과 직위는 중요하다. 더러운 물질만능주의 세상(의역 ㅎㅎ)

조금 더 파해쳐보고 다른 흥미로운 관측치들을 확인하자. 한 번 **Sex와 Pclass**의 생존률을 한번에 보자.

In [ ]:
pd.crosstab([data.Sex,data.Survived],data.Pclass, margins = True).style.background_gradient(cmap='summer_r')

In [ ]:
sns.factorplot('Pclass', 'Survived', hue = 'Sex', data = data)
plt.show()

categorical values 쉽게 나눠 주기 때문에 여기서는 **FactorPlot** 를 사용하였다.

**CrossTab** 과 **FactorPlot**를 보면 **Women from Pclass1**의 생존률은 약 **95-96%** 되는것을 확인할 수 있다. 오직  Pclass1의 여자 94명중 3명만이 사망하였다. 

Pclass와는 상관없이 여자가 먼저 구조된것을 확인 할 수 있었다. 심지어 Pclass1의 남자도 매우 낮은 생존률을 보인다.

Pclass도 중요한 변수처럼 보인다. 이제 다른 feature을 분석해보자.

## Age--> Continous Feature


In [ ]:
print('Oldest Passenger was of:',data['Age'].max(),'Years')
print('Youngest Passenger was of:',data['Age'].min(),'Years')
print('Average Age on the ship:',data['Age'].mean(),'Years')

In [ ]:
f,ax = plt.subplots(1,2, figsize = (18,8))
sns.violinplot('Pclass', 'Age', hue = "Survived", data = data, split=True, ax= ax[0])
ax[0].set_title("Pclass and Age vs Survived")
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex", 'Age', hue = "Survived", data = data, split = True, ax = ax[1])
ax[1].set_title("Sex and Age vs Survived")
ax[1].set_yticks(range(0,110,10))
plt.show()

#### Observations:

1)Pclass에 따라 아이들의 수는 증가하고 10살 이하의 승객들의 생존률은 Pclass와는 상관없어 보인다.

2)Pclass1의 20-50대 승객들을 높은 생존확률을 보이고 심지어 여자보다도 높다.

3)남성들은 나이가 많아짐에 따라 생존확률은 감소한다.
For males, the survival chances decreases with an increase in age.

사전에 봤던것처럼, Age feature는 **177** 개의 결측치를 가지고있다. 이를 대체하기 위해서 data set의 평균 나이를 할당할 것이다.

그러나 문제는 나이가 다른 많은 사람들이 있다는 것이다. 4살짜리 아이에게 평균나이인 29살을 할당 할 수 없기에 승객이 어떤 나이대에 속할지 다른 방법은 없는지 생각해보자.

**Bingo!!!!**, 여기서 **Name** feature를 확인 할 수 있다. 이 feature를 살펴보면 name에는 Mr or Mrs 와 같은 a salutation 을 가지고 있는것을 볼 수 있다. 그러므로 각각의 그룹의 Mr 그리고 Mrs의 평균값을 할당할 것이다.

**''What's In A Name??''**---> **Feature**  :p

In [ ]:
data.Name

In [ ]:
data['Initial']=0
for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
    # '([A-Za-z]+)\.' 정규표현식

여기서 Regex(정규표현식): : **[A-Za-z]+)\.**. 을 사용하였다. 그것이 하는것은 사이에 있는 문자열을 찾고 그 뒤를 **.(dot)** 가 뒤 따른다. 완벽하게 Name으로 부터 Initials를 추출해 냈다.

In [ ]:
pd.crosstab(data.Initial,data.Sex).T.style.background_gradient(cmap='summer_r') #Checking the Initials with the Sex

잘목 적힌것 같은 Mlle 그리고 Mme가 있었는데 이건 Miss를 말하는것 같다. 이것들을 Miss 로 대체하고 똗같은 것들을 다른 값으로 대체할것이다. 

In [ ]:
data['Initial'].replace(["Mlle","Mme",'Ms',"Dr","Major","Lady","Countess","Jonkheer",'Col',"Rev",'Capt',"Sir","Don", 'Dona'],
                        ["Miss","Miss","Miss", "Mr", 'Mr', "Mrs","Mrs", "Other", "Other", 'Other', 'Mr', 'Mr', 'Mr', 'Other'], inplace = True)

In [ ]:
data.groupby('Initial')['Age'].mean() #lets check the average age by Initials

### Filling NaN Ages

In [ ]:
## Assigning the NaN Values with the Ceil values of the mean ages
data.loc[(data.Age.isnull())&(data.Initial=='Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.Initial=='Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.Initial=='Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.Initial=='Other'),'Age']=46

In [ ]:
data.Age.isnull().any() #So no null values left finally 

In [ ]:
f,ax = plt.subplots(1,2, figsize = (20,10))
data[data['Survived']== 0].Age.plot.hist(ax = ax[0], bins = 20, edgecolor = 'black', color = 'red')
ax[0].set_title('Survived = 0')
x1 = list(range(0, 85, 5))
ax[0].set_xticks(x1)
data[data['Survived']==1].Age.plot.hist(ax=ax[1], color = 'green', bins = 20, edgecolor = 'black')
ax[1].set_title('Survived = 1')
x2 = list(range(0,85,5))
ax[1].set_xticks(x2)
plt.show()

### Observations:
1)갓난아기(AGE<5)는 대다수가 살아남았다(The Women and Child First Policy).

2)가장 나이많은 승객은 살아남았다(80세)

3)사망의 가장많은 수는 30~40대의 그룹에 속한다.

In [ ]:
sns.factorplot('Pclass', 'Survived', col = 'Initial', data = data)
plt.show()

class에 상관없이 여성과 아이를 우선적으로 구했다.

## Embarked--> Categorical Value

In [ ]:
pd.crosstab([data.Embarked,data.Pclass],[data.Sex, data.Survived], margins= True).style.background_gradient(cmap="summer_r")

### Chances for Survival by Port Of Embarkation

In [ ]:
sns.factorplot('Embarked', 'Survived',data =data)
fig=plt.gcf() #figure 에 접근
fig.set_size_inches(5,3)
plt.show()

port C에 대한 생존확률이 0.55로 가장 높고 S에서 가장 낮다.

In [ ]:
f, ax = plt.subplots(2,2,figsize=(20,15))
sns.countplot('Embarked', data = data, ax = ax[0,0])
ax[0,0].set_title('No. Of Passengers Boarded')
sns.countplot('Embarked', hue = 'Sex', data = data, ax = ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked')
sns.countplot('Embarked', hue = 'Survived', data =data, ax = ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot('Embarked', hue = 'Pclass', data=data, ax = ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2,hspace=0.5) # 그래프간 간격 조정 width height
plt.show()

### Observations:
1)다수의 승객이 S에서 탑승했다. 그들 중 다수가 Pclass3 이다.

2)C에서 탑승한 승객들은 생존률을 보면 상당히 운이 좋은것 처럼 보인다. 이에 대한 이유는 아마도 Pclass1 과 Pclass2의 모든(다수) 승객들이 구조되서 그럴것이다.

3)The Embark S 대다수의 부자 사람들이 탑승한 port처럼 보인다. 그러나 생존률은 여전히 낮은데 아마도 약 **81%** Pclass3 사람들이 살아남지 못했기 때문이다.

4)Port Q는 거의 95%의 승객이 Pclass3로부터 왔다.

In [ ]:
sns.factorplot('Pclass', 'Survived', hue = 'Sex', col = 'Embarked', data = data)
plt.show()

### Observations:

1)Embarked와 상관없이 Pclass1 과 Pclass2 여성의 생존확률은 거의 1이다.

2)여성과 남성 모두 생존률이 매우 낮기 때문에 port S는 Pclass3 승객에게는 매우 불행한것 처럼 보인다.**(Money Matters)**

3)port Q는 남자에게 매우 불행한것처럼 보인다. 왜냐면 거의 모든사람이 Pclass3출신이기 때문이다.

### Filling Embarked NaN

제일많은 승객들이 port S에서 탑승했기 때문에 NaN값을 S로 대체한다.

In [ ]:
data['Embarked'].fillna('S',inplace=True)

data.Embarked.isnull().any()# 마침내 결측치가 존재하지 않는다.

## SibSip-->Discrete Feature
이 feature는 홀로 사는지 아니면 가족과 같이 사는지를 나타낸다.

Sibling = brother, sister, stepbrother, stepsister

Spouse = husband, wife 

In [ ]:
pd.crosstab(data.SibSp, data.Survived).style.background_gradient(cmap='summer_r')

In [ ]:
f,ax = plt.subplots(1,2, figsize = (20, 8))
sns.barplot('SibSp', 'Survived',data =data, ax = ax[0])
ax[0].set_title('SibSp vs Surivved')
sns.factorplot('SibSp', 'Survived', data = data, ax = ax[1])
ax[1].set_title('SibSp vs Survived')
plt.close(2)
plt.show()

In [ ]:
pd.crosstab(data.SibSp,data.Pclass).style.background_gradient(cmap='summer_r')

### Observations:


The barplot and factorplot에 의하면 만약 승객이 sibling 없이 혼자 탑승했다면 34.5%의 생존률을 보였다. 그래프는 대략 sibling의 수가 증가함에 따라 생존률이 감소한다. 이건 이해가 된다. 즉 한 사람이 가족과 함께 탑승 했다면 내 가족을 구하기 위해 자신을 희생할 것이다. 놀랍게도 5-8명의 가족들의 생존률은 **0%** 보인다. 이유는 아마도 Pclass일겁니다???

이유는 **Pclass** 이다. crosstab을 보면 SibSo>3 인 사람들은 모두 Pclass3dp 속해있다. Pclass3에 있는 대 가족들은 모두 사망했다는것이 분명하다.

## Parch

In [ ]:
pd.crosstab(data.Parch,data.Pclass).style.background_gradient(cmap='summer_r')

다시 crosstab에 의하면 대가족들은 Pclass3에 속한다.

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,8))
sns.barplot('Parch','Survived',data=data,ax=ax[0])
ax[0].set_title('Parch vs Survived')
sns.factorplot('Parch','Survived',data=data,ax=ax[1])
ax[1].set_title('Parch vs Survived')
plt.close(2)
plt.show()

### Observations:

여기 매우 유사한 결과들이 보인다. 부모와 함께 탑승한 승객들은 높은 생존확률을 보였다. 그러나 수가 커질수록 생존률이 감소하였다.

생존률은 1-3 parents가 배에 탑승한 사람들이 높았다. 혼자 있다는것은 치명적이고 >4 parents를 가진 사람들은 생존률이 감소하였다.

## Fare--> Continous Feature

In [ ]:
print('Highest Fare was:',data['Fare'].max())
print('Lowest Fare was:',data['Fare'].min())
print('Average Fare was:',data['Fare'].mean())

In [ ]:
data.Fare.fillna(data.groupby(['Pclass', 'Sex']).Fare.transform('median'), inplace = True)

가장 낮은 탑승요금은 **0.0**입니다. Wow!! a free luxorious ride. 

In [ ]:
f,ax=plt.subplots(1,3,figsize=(20,8))
sns.distplot(data[data['Pclass']==1].Fare, ax =ax[0])
ax[0].set_title('Fares in Pclass 1')
sns.distplot(data[data['Pclass']==2].Fare, ax = ax[1])
ax[1].set_title('Fare in Pclass 2')
sns.distplot(data[data['Pclass']==3].Fare,ax=ax[2])
ax[2].set_title('Fares in Pclass 3')
plt.show()

Pclass1 의 승객의 요금에는 a large distribution 이 있는게 보인다 그리고 이 distribution은 standards 가 감소함에 따라 감소한다. Fare또한 연속형이기 떄문에 binning을 통해 discrite하게 만들 수 있다.

## Observations in a Nutshell for all features:
**Sex:** 남성과 비교하면 여성의 생존률이 높다.

**Pclass:**1st calss 의 승객들이 더 높은 생존률을 보였다. Pclass3 승객들의 생존률을 매우 낮았다. **여성들의 경우에**, **Pclass1** 의 승객들의 생존률은 거의 1에 가까웠고 **Pclass2** 승객들 또한 매우 높았다. **돈이 최고다!!!**.

**Age:** 5-10살의 아이들은 매우 높은 생존률을 보였다. 15-35세 사이의 승객들은 많이 사망했다.

**Embarked:** 이건 매우 흥미로운 feature이다. **대다수의 Pclass1 의 승객이 S에서 탑승했음에도  C에서의 생존률은 좋아 보인다.** Q에서의 승객들은 모두 **Pclass3**이다.

**Parch+SibSp:** 1-2명의 형제 자매나 배우자 또는 1-3명의 부모님과 같이 탑승한 승객은 혼자 탑승하거나 큰 가족 단위로 여행하는 승객보다 더 높은 생존률을 보인다.

## Correlation Between The Features

In [ ]:
sns.heatmap(data.corr(), annot=True, cmap = 'RdYlGn', linewidth = 0.2)#data.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

### Interpreting The Heatmap

먼저 주목할 점은 알파벳이나 문자열은 상관계수를 구할 수 없는것이 명백하기 때문에 숫자 feature들만 비교됬다는 것이다. plot을 이해하기전에 correlation이 정확히 뭔지 한 번 보자.

**POSITIVE CORRELATION:** 만약 **feature A의 증가가 feature B의 증가로 이어진다면 두 변수 사이는 positively correlated** 이다. 값이 **1이라는 건 완벽한 양의 상관관계를 의미한다**.

**NEGATIVE CORRELATION:** 만약 **feature A의 증가가 Feature B의 감소로 이어진다면 두 변수는 negatively correlated 되어있다**. 값이 **-1이라는건 완벽한 음의 상관관계라는것을 의미한다. (negative correlation)**.

2개의 features가 높게 correlated 되어있다면 한 feature 증가는 다른 feature 증가로 이어질것이다. 이것은 2개의 feature 가 유사한 정보를 담고있고 매우 적은 정보안에 분산을 가지고 있다는 것을 의미한다. 이것은 **다중공선성** 으로 알려져 있으며 그 두개의 변수가 거의 동일한 정보를 가지고있는 것이다.

그러면 우리가 그것들중 하나의 변수가 쓸모없음에도 그 두변수 모두 사용해야할까? 트레이닝을 만드는 동안 우리는 그 불필요한 feature를 제거한후에 그것이 trainin time을 감소시키고 많은 이점이 있는지 확인해야 한다.

위의 heatmap을 보건데 feature간에 상관관계는 많지 않은것을 볼 수 있다. 가장 높은 correlationdms **SibSp 와 Parch 즉 0.41** 정도이므로 우리는 모든 feature를 사용할 것이다.

## Part2: Feature Engineering and Data Cleaning

Feature Engineering이란 무엇일까?

우리가 feature와 dataset이 주어졌을때 모든 feature가 중요하지는 않을것이다. 아마도 제거되어야 할 불필요한 features들이 존재할 것이다. 또한 다른 feature로 부터 정보를 얻어서 새로운 feature를 생성할 수 있다.

한 예시로는 Initial feature를 생성한것이다 Name feature를 사용하여서. 우리가 새로운 feature를 얻거나 제거할 수 있는지 확인해 보자. 또한 존재하는 연관된 feature를 modeling에 적합한 형태로 바꿀것이다.

## Age_band

#### Problem With Age Feature:
앞서 언급하였다 싶이 **Age는 연속형 feature**인데 머신러닝 모델에 연속형 변수와 관련한 문제가 있다.

**Eg:**만약 내가 운동 선수를 group을 만들거나 정렬을 해야한다면 **성별** 을 이용할 수 있다. 즉 남성과 여성으로 쉽게 구분지을 수 있다.

만약 이제 **나이** 로 그들을 그룹을 지어야 한다면, 어떻게 할 것인가? 만약 30명의 사람이 있다면 30개의 나이값이 있을것이다. 이제 이건 문제를 야기시킨다.

우리는 이제 이 **연속형변수를 binning과 nomalisation을 통해 범주형 변수로** 바꿔줄 필요가 있다. 여기서는 binning 즉 age 범위를 그룹화 시켜주는것을 통해 하나의 구간 또는 그것들의 하나의 값을 할당해줄 것이다.

승객의 최대 나이는 80세이니 따라서 0-80나의를 5개의 구간으로 나누자. 
즉 80/5 = 16이므로 구간의 크기는 16이다.

In [ ]:
data['Age_band']=0
data.loc[data['Age']<=16,'Age_band']=0
data.loc[(data['Age']>16)&(data['Age']<=32),'Age_band']=1
data.loc[(data['Age']>32)&(data['Age']<=48),'Age_band']=2
data.loc[(data['Age']>48)&(data['Age']<=64),'Age_band']=3
data.loc[data['Age']>64,'Age_band']=4
data.head(2)

In [ ]:
data['Age_band'].value_counts().to_frame().style.background_gradient(cmap='summer')#checking the number of passenegers in each band

In [ ]:
sns.factorplot('Age_band','Survived',data=data,col='Pclass')
plt.show()

Pclass와 상관없이 나이가 증가함에 따라 생존률이 감소하는것을 볼 수 있다.

## Family_Size and Alone
이시점에서 "Family_size"와 "Alone"이라는 새로운 feature만들어 그것을 분석 할 수 있다. 이 Feature는 Parcg와 SibSp의 합이다. 이건 combined(합쳐진?) Data를 제공해서 생존률이 승객의 가족의 크기에 영향이 있는지 확인 할 수 있게 한다. Alone은 승객이 혼자인지 아닌지를 말해주는 것이다.

In [ ]:
data['Family_Size']=0
data['Family_Size']=data['Parch']+data['SibSp']#family size
data['Alone']=0
data.loc[data.Family_Size==0,'Alone']=1#Alone

f,ax = plt.subplots(1,2, figsize=(18,6))
sns.factorplot('Family_Size', 'Survived', data = data, ax =ax[0])
ax[0].set_title('Family_Size vs Survived')
sns.factorplot('Alone', 'Survived', data = data, ax = ax[1])
ax[1].set_title('Alone vs Survived')
plt.close(2)
plt.close(3)
plt.show()

**Family_Size=0 건 승객이 혼자 탑승했다는것을 의미한다.** 명확하게 만약 혹자 탑승했거나 family_size=0이라면 생존확률이 매우 낮다. Family_size>4인 사람들 또한 매우 낮은 생존률을 보인다. 이것 또한 model에서 중요한 feature라고 생각되어진다. 조금 더 조사해보자.

In [ ]:
sns.factorplot('Alone','Survived', hue = 'Sex', col = 'Pclass', data = data)
plt.show()

혼자인 여성의 생존률이 가족이 있는 여성보다 더 높은 Pclass3를 제외하면 성별이나 Pclass 를 막론하고 혼자있다는것은 매우 낮은 생존률을 보인다.

## Fare_Range

요금은 중요한 연속형 feature이기 때문에, 우리는 이것을 ordinal 값으로 바꿔야한다. 이것을 위해 **pandas.qcut** 을 사용할 것이다.

**qcut**은 입력한 구간의 값에 따라 값을 나누고 정렬한다. 만약 우리가 5개의 구간을 입력했다면 그것은 동일한 크기의 구간으로 값을 정렬시킬것이다.

In [ ]:
data['Fare_Range']=pd.qcut(data['Fare'],4)
data.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

위에서 논의되었던 것처럼, **fare_range가 올라감에따라 생존률도 올라가는것** 을 확인할 수 있다.

Fare_Range 값을 그대로 입력할 수 없으니 **Age_Band**에서 했던것처럼 구간을 하나의 값으로 바꿔줄 것이다.

In [ ]:
data['Fare_cat']=0
data.loc[(data.Fare<=7.91),'Fare_cat'] = 0
data.loc[(data['Fare']>7.91)&(data['Fare']<=14.454),'Fare_cat'] = 1
data.loc[(data['Fare']>14.454)&(data['Fare']<=31),'Fare_cat'] = 2
data.loc[(data['Fare']>31),'Fare_cat'] = 3

sns.factorplot('Fare_cat','Survived',data=data,hue='Sex')
plt.show()

명확하게 Fare_cat이 증가함에따라 생존률도 증가하는것이 보인다. 이 feature는 Sex와 함께 모델링 할때 중요한feature될것이다.


## Converting String Values into Numeric

우리는 머신러닝 model에 문자열을 전달 할 수 없기에 Sex, Embarked와 같은 feature를 numeric value로 바꿔줘야한다.

In [ ]:
data['Sex'].replace(['male','female'],[0,1],inplace = True)
data['Embarked'].replace(['S','C','Q'],[0,1,2], inplace = True)
data['Initial'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4], inplace =True)

### Dropping UnNeeded Features

**Name**--> 이미 범주형변수(categorical value)값으로 변환하였기 때문에 더이상 name feature는 필요없다.

**Age**--> Age_band feature가 있으므로 이것도 더이상 필요없다.

**Ticket**--> 이것은 categorised(범주화)될수 없는 무작위 문자이다.

**Fare**--> Fare_cat feature를 가지고 있으므로 필요없다.

**Cabin**--> 많은 결측값과 또한 많은 승객이 다수의 cabin을 가지고 있으므로 이것또한 필요없는 변수이다.

**Fare_Range**--> fare_cat feature가 있다.

**PassengerId**--> 범주화 될수 없다..

In [ ]:
data.drop(['Name', 'Age', 'Ticket', 'Fare', 'Cabin', 'Fare_Range', 'PassengerId'], axis = 1, inplace =True)
sns.heatmap(data.corr(),annot = True, cmap = 'RdYlGn', linewidths = 0.2, annot_kws = {'size':20})
fig=plt.gcf()
fig.set_size_inches(18,15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

위에 correlation plot을 보면 우리는 몇개의 양의 상관관계를 가진 feature들을 볼 수 있다. 양의 상관관계는 **SibSp andd Family_Size** and **Parch and Family_Size** 이고 음의 상관관계는 **Alone and Family_Size.** 이다.

In [ ]:
df_test = data[data['Survived'].isnull()].drop('Survived', axis = 1)

In [ ]:
data = data.dropna(subset = ['Survived'])

In [ ]:
#importing all the required ML packages
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix

In [ ]:
train,test=train_test_split(data,test_size=0.3,random_state=0,stratify=data['Survived'])
train_X=train[train.columns[1:]]
train_Y=train[train.columns[:1]]
test_X=test[test.columns[1:]]
test_Y=test[test.columns[:1]]
X=data[data.columns[1:]]
Y=data['Survived']

### Radial Support Vector Machines(rbf-SVM)

In [ ]:
model=svm.SVC(kernel='rbf',C=1,gamma=0.1)
model.fit(train_X,train_Y)
prediction1=model.predict(test_X)
print('Accuracy for rbf SVM is ',metrics.accuracy_score(prediction1,test_Y))

### Linear Support Vector Machine(linear-SVM)

In [ ]:
model=svm.SVC(kernel='linear',C=0.1,gamma=0.1)
model.fit(train_X,train_Y)
prediction2=model.predict(test_X)
print('Accuracy for linear SVM is',metrics.accuracy_score(prediction2,test_Y))

### Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(train_X,train_Y)
prediction3=model.predict(test_X)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(prediction3,test_Y))

### Decision Tree

In [ ]:
model=DecisionTreeClassifier()
model.fit(train_X,train_Y)
prediction4=model.predict(test_X)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction4,test_Y))

### K-Nearest Neighbours(KNN)

In [ ]:
model=KNeighborsClassifier() 
model.fit(train_X,train_Y)
prediction5=model.predict(test_X)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction5,test_Y))

KNN 모델에대한 정확도는 **n_neighbours**의 값을 바꿀때마다 따라 달라진다. 초기값은 **5** 로 설정 돼있다.
n_neighbours에 따른 정확도를 확인해보자.

In [ ]:
a_index=list(range(1,11))
a=pd.Series()
x=[0,1,2,3,4,5,6,7,8,9,10]
for i in list(range(1,11)):
    model=KNeighborsClassifier(n_neighbors=i) 
    model.fit(train_X,train_Y)
    prediction=model.predict(test_X)
    a=a.append(pd.Series(metrics.accuracy_score(prediction,test_Y)))
plt.plot(a_index, a)
plt.xticks(x)
fig=plt.gcf()
fig.set_size_inches(12,6)
plt.show()
print('Accuracies for different values of n are:',a.values,'with the max value as ',a.values.max())

### Gaussian Naive Bayes

In [ ]:
model=GaussianNB()
model.fit(train_X,train_Y)
prediction6=model.predict(test_X)
print('The accuracy of the NaiveBayes is',metrics.accuracy_score(prediction6,test_Y))

### Random Forests

In [ ]:
model=RandomForestClassifier(n_estimators=100)
model.fit(train_X,train_Y)
prediction7=model.predict(test_X)
print('The accuracy of the Random Forests is',metrics.accuracy_score(prediction7,test_Y))

모델의 정확도만 classifier의 robustness(안정도?)를 결정하는 것은 아니다. classifer 가 traing 되고 test 했을때 정확도가 90%를 기록했다고 하자.

이것은 성능좋은 classifier로 보이지만, 그러나 새로운 test set에 test됬을때 정확도가 90%거라고 말할 수 있을까? 답은 **그렇지 않다** 이다, 왜냐면 우리는 classifier가 어떤 instance를 사용해서 학습될지 결정 할 수 없기 때문이다. 트레에닝 data와 test데이터가 바뀜에 따라 정확도 또한 달라질것이다. 그것은 증가할 수도 또는 감소할 수도 있다. 이것은 **model variance** 라고 알려져 있다.

이것을 극복하기위해 좀더 일반화된 model을 얻는데, **Cross Validation**을 사용한다.

# Cross Validation

많은 경우에 데이터가 불균형하다. 즉 class1의 instance는 많이 존재하지만 다른 class의 수는 적다. 그러므로 우리는 반드시 데이터셋의 모든 instance에 train 하고 test하여야 한다. 그러면 우리는 데이터셋에 대하여명시된 정확도의 평균을 얻을 수 있다. 

1)The K-Fold Cross Validation은 k개의 subset(부분집합)으로 데이터셋을 나눈것으로 작동한다.

2)데이터셋을 (k=5) 부분으로 나눈다고 하자. 우리는 한개의 부분을 테스트를 위해 남겨주독 4개의 부분을 알고리즘을 통해 학습시킨다.

3)각각의 반복에서 테스트하는 부분을  바꾸고 나머지 부분으로 알고리즘을 training 함으로서 이 과정을 계속한다. 정확도와 오차를 평균을 내면 알고리즘의 평균적인 정확도를 알 수 있다.

이것을 k-fold Cross Validation이라 부른다.

4)한 알고리즘은 어떠한 training data에 대해서는 underfit 될 수 있고 때때로 다른 training data에 대해 overfit 될 수도있다. 그러므로 cross-validation을 통해 좀 더 일반적인 모델을 얻을 수 있다.

In [ ]:
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
xyz=[]
accuracy=[]
std=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree','Naive Bayes','Random Forest']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=9),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(n_estimators=100)]
for i in models:
    model = i
    cv_result = cross_val_score(model,X,Y, cv = kfold,scoring = "accuracy")
    cv_result=cv_result
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std},index=classifiers)       
new_models_dataframe2

In [ ]:
plt.subplots(figsize=(12,6))
box=pd.DataFrame(accuracy,index=[classifiers])
box.T.boxplot()

In [ ]:
new_models_dataframe2['CV Mean'].plot.barh(width=0.8)
plt.title('Average CV Mean Accuracy')
fig=plt.gcf()
fig.set_size_inches(8,5)
plt.show()

불균등성 때문에 classification accuracy는 때때로 잘못 해석(misleading)될 수도 있다. confution matrix를 통해 더 종합적인 결과를 얻을 수 있는데 이것이 모델이 어디로 잘못 갔는지 또는 어떤 class를 모델이 잘못 예측했는지 알려줄 것이다.

# Confusion Matrix

classifier에 의해 예측된 맞은 예측과 틀린 예측의 수를 보여준다.

In [ ]:
f,ax=plt.subplots(3,3,figsize=(12,10))
y_pred = cross_val_predict(svm.SVC(kernel='rbf'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,0],annot=True,fmt='2.0f')
ax[0,0].set_title('Matrix for rbf-SVM')
y_pred = cross_val_predict(svm.SVC(kernel='linear'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,1],annot=True,fmt='2.0f')
ax[0,1].set_title('Matrix for Linear-SVM')
y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors=9),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,2],annot=True,fmt='2.0f')
ax[0,2].set_title('Matrix for KNN')
y_pred = cross_val_predict(RandomForestClassifier(n_estimators=100),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,0],annot=True,fmt='2.0f')
ax[1,0].set_title('Matrix for Random-Forests')
y_pred = cross_val_predict(LogisticRegression(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,1],annot=True,fmt='2.0f')
ax[1,1].set_title('Matrix for Logistic Regression')
y_pred = cross_val_predict(DecisionTreeClassifier(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,2],annot=True,fmt='2.0f')
ax[1,2].set_title('Matrix for Decision Tree')
y_pred = cross_val_predict(GaussianNB(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[2,0],annot=True,fmt='2.0f')
ax[2,0].set_title('Matrix for Naive Bayes')
plt.subplots_adjust(hspace=0.2,wspace=0.2)
plt.show()

### Interpreting Confusion Matrix

왼쪽의 대각선은 각각의 class에 대하여 올바르게 예측한 경우의 수를 의미하고 오른쪽 대각선의 경우는 잘못 예측한 경우의 수이다. 첫번째 그래프인 rbf-SVM을 살펴보자:

1)옳은 예측의 수는 **491(사망) + 247(생존)** 이고 평균적인 CV 정확도는 **(491+247)/891 = 82.8 %** 을 보였다.

2)**Errors**-->  58명의 사망자들은 생존으로 95명의 생존자들은 사망했다고 잘못 예측되었다. 그러므로 살아남은 사람들을 사망헀다고 예측하는 잘못 예측하는 일이 더 많았다.

모든 metrices을 보면 rbf-SVM이 가장 사망자를 잘예측하였으며 NaiveBayes의 경우 생존자를 가장 잘 예측하였다.


### Hyper-Parameters Tuning

머신러닝 모델의 경우 블랙박스와 같다. 이 블랙박스의 경우 parameter의 초기값이 있는데 더 나은 모델을 얻기 위해 parameter를 tune 하고 change 하여야 한다. SVM 모델의 C 와 gamma 처럼 다른 classifiers에 대해 다른 parameter들이 존재하고 이것들을 hyper-parameter라고 부른다. hyper-parameter를 tune하고 알고리즘의 learning-rate를 바꾸면서 우리는 더 나은 모델을 얻을 수 있다. 이것은 Hyper-parameter Tuning 이라고 부른다.

여기서는 2개의 가장 좋은 모델인 SVM과 RandomForest를 hyper - parameter tunning 할 것이다.

#### SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
C=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
gamma=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
kernel=['rbf','linear']
hyper={'kernel':kernel,'C':C,'gamma':gamma}
gd=GridSearchCV(estimator=svm.SVC(),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

### Random Forests

In [ ]:
n_estimators=range(100,1000,100)
max_depth = range(2,8)
hyper={'n_estimators':n_estimators, 'max_depth' : max_depth}
gd=GridSearchCV(estimator=RandomForestClassifier(random_state=0),param_grid=hyper,cv=5,verbose=True, n_jobs=4)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

Rbf-SVM의 최고 score는 **82.82%고 C = 0.5 그리고 gamma = 0.1 이다.**
RandomForest의 경우 **83.27%이고 n_estimators = 100 그리고 max_depth = 4 이다.**

# Ensembling

앙상블은 model의 성능과 정확도를 올리기위한 좋은 방법이다. 간단하게 말하면 이것은 하나의 강력한 model을 만들기 위해 다양한 간단한 모델을 조합하는것을 의미한다. 

만약 우리가 핸드폰을 구매하려고하고 많은 사람들에게 다양한 조건에 기반하여 물어봤다고 하자. 다른 조건들을 분석한 후에 우리는 한 상품에 대해 더 확신에 찬 판단을 만들 수 있다. 이것이 **EnSembling** 이다. 앙상블은 다음과 같은 방법으로 된다:

1)Voting Classifier

2)Bagging

3)Boosting.

## Voting Classifier

이것은 은 다른 간단한 머신러닝 모델로 부터의 예측을 조합하는 가장 간단한 방법이다. 이것은 기반이 되는 모든 모델의 예측에 근거하여 평균적인 예측결과를 제공한다. 기반이 되는 모델은 모두 다른 타입이다.

In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble_lin_rbf=VotingClassifier(estimators=[('KNN',KNeighborsClassifier(n_neighbors=10)),
                                              ('RBF',svm.SVC(probability=True,kernel='rbf',C=0.5,gamma=0.1)),
                                              ('RFor',RandomForestClassifier(n_estimators=100,random_state=0, max_depth = 4)),
                                              ('LR',LogisticRegression(C=0.05)),
                                              ('DT',DecisionTreeClassifier(random_state=0)),
                                              ('NB',GaussianNB()),
                                              ('svm',svm.SVC(kernel='linear',probability=True))
                                             ], 
                       voting='soft').fit(train_X,train_Y)
print('The accuracy for ensembled model is:',ensemble_lin_rbf.score(test_X,test_Y))
cross=cross_val_score(ensemble_lin_rbf,X,Y, cv = 10,scoring = "accuracy")
print('The cross validated score is',cross.mean())

In [ ]:
a = ensemble_lin_rbf.predict(df_test)

In [ ]:
pd.concat(["cc"=a, "dasdaasd"= a], axis = 1)

In [ ]:
pd.concat([test_id, pd.Series(a)], axis = 1).to_csv('SUBMIT.csv', index=False)

## Bagging

배깅은 일반적인 앙상블 방법이다. 이것은 유사한 classifiers를 데이터셋의 작은 부분에 적용하고 모든 예측의 평균을 내는것이다. 평균을 내는 덕분에 분산을 감소시킨다. Voting Classifier과는 다르게 Bagging은 유사한 classifier들만 이용한다.

#### Bagged KNN

배깅은 높은 분산을 가진 모델에 가장 잘 작동한다. 이것의 예시로는 Decision Tree또는 Random Forests를 들 수 있다.
여기서는 작은 **n_neighbours** 값을 가지는 KNN을 사용할 것이다.

In [ ]:
from sklearn.ensemble import BaggingClassifier
model=BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=3),random_state=0,n_estimators=700)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged KNN is:',metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for bagged KNN is:',result.mean())

#### Bagged DecisionTree

In [ ]:
model=BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=0,n_estimators=100)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged Decision Tree is:',metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for bagged Decision Tree is:',result.mean())

## Boosting

부스팅은 연속적인 classifiere의 학습을 사용하는 앙상블이다. 이것은 약한 모델의 단계적인 강화과정으로 부스팅이 작동하는과정은 다음과 같다:

모델은 우선 전체 데이터셋에 대하여 학습된다. 그러면 어떤 instance들은 올바르게 예측된 반면에 어떤것들은 잘못 예측됬을 것이다. 다음의 학습서 모델은 잘못 예측된 instance 에 집중하거나 또는 그것들에 대하여 더 가중치를 줄것이다. 그러면 잘못 예측된것들을 올바르게 예측하도록 시도 할것이다. 이제 이 과정을 계속해서 정확도의 한계에 다다를때 까지 새로운 classifier를 모델에 더할것이다.

#### AdaBoost(Adaptive Boosting)

이 경우에 약한 learner or estimator(모델)은 Decision Tree 였으나 선택의 따라 초기 base_estimator을 바꿀 수 있다. 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1)
result=cross_val_score(ada,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for AdaBoost is:',result.mean())

#### Stochastic Gradient Boosting

여가도 또한 weak learner 로 Decision Tree를 사용하였다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
grad=GradientBoostingClassifier(n_estimators=500,random_state=0,learning_rate=0.1)
result=cross_val_score(grad,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for Gradient Boosting is:',result.mean())

#### XGBoost

In [ ]:
import xgboost as xg
xgboost=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
result=cross_val_score(xgboost,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for XGBoost is:',result.mean())

가장 높은 정확도는 AdaBoost 였고 Hyper-Parameter Tuning을 통해 정확도를 향상시킬것이다.

#### Hyper-Parameter Tuning for AdaBoost

In [ ]:
n_estimators=list(range(100,1100,100))
learn_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
gd=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=hyper,verbose=True, njob)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

AdaBoost를 이용해서 얻은 최적의 정확도는 **83.16% 이고 n_estimators=200 and learning_rate=0.05이다.**

### Confusion Matrix for the Best Model

In [ ]:
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.05)
result=cross_val_predict(ada,X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,result),cmap='winter',annot=True,fmt='2.0f')
plt.show()

## Feature Importance

In [ ]:
f,ax=plt.subplots(2,2,figsize=(15,12))
model=RandomForestClassifier(n_estimators=500,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,0])
ax[0,0].set_title('Feature Importance in Random Forests')
model=AdaBoostClassifier(n_estimators=200,learning_rate=0.05,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,1],color='#ddff11')
ax[0,1].set_title('Feature Importance in AdaBoost')
model=GradientBoostingClassifier(n_estimators=500,learning_rate=0.1,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,0],cmap='RdYlGn_r')
ax[1,0].set_title('Feature Importance in Gradient Boosting')
model=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,1],color='#FD0F00')
ax[1,1].set_title('Feature Importance in XgBoost')
plt.show()

RandomForests, AdaBoost 등등.. 과 같은 다양한 classifiers의 feature importance를 확인 할 수 있다.

#### Observations:

1)일반적으로 중요한 features 는 Initial,Fare_cat,Pclass,Family_Size이다.

2)Sex feature의 경우 어떠한 중요도를 보이지 않았다 이것은 충격적이다 왜냐면 우리는 전에 확인했기 때문이다 Pclss와 함께 Sex 는 확연한 차이를 나타내는 factor였기 때문이다. Sex는 오직 RandomForests에서만 중요한것처럼 보인다.

그러나 feature Initial을 보면, 이건 많은 classifiers에서 순위권에 위치하고 있다. 이미 Sex 와 Initial 과의 양의 상관 관계를 확인했기 때문에 두개다 모두 성별을 의미한다고 할 수 있다.

3)유사하게 Pclass 와 Fare_cat은 승객의 지위(?)를 나타내고 Family_Size 도 다른변수 Alone,Parch and SibSp과 유사하다.

여러분 모두가 머신러닝에 관해 뭔가 알아갔기를 희망합니다. 머신러닝에 관한 다른 훌륭한 notebook 들은 다음과 같습니다.
1) For R:[Divide and Conquer by Oscar Takeshita](https://www.kaggle.com/pliptor/divide-and-conquer-0-82297/notebook)

2)For Python:[Pytanic by Heads and Tails](https://www.kaggle.com/headsortails/pytanic)

3)For Python:[Introduction to Ensembling/Stacking by Anisotropic](https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python)

### 이 글을 봐주셔서 감사합니다 . 만약 이 글이 도움이 되었다면 [https://www.kaggle.com/ash316/eda-to-prediction-dietanic]들어가셔서 **Upvote를 눌러주세요**

### I'm really appreciate Writer of this original kernel. i've learn so many things and Those are so helpful for me. 그러니 이 한글번역이 도움위 되셨다면 링크 들어가셔서 Upvote 한 번만 눌러주세요 감사합니다.